In [81]:
import numpy as np
from src.common.fitness import griewank, rastrigin, ackley
from src.gendered_selection.helpers import (calculate_lifetime)
from src.gendered_selection.conf.gendered_selection_config import Config
from src.gendered_selection.faster_fuzzy_logic.infer_partner_age import Inferrer
import multiprocessing as mp

In [89]:
population_scale = 1
N =  1000 
FS =  Inferrer(rule_path=Config.RULES_FILE)

genomes = np.random.uniform(-population_scale, population_scale, size=(N, 5))
fitness = np.apply_along_axis(griewank, 1, genomes)
gender = (np.random.rand(N) >= .5).astype(int)
age = np.zeros(N)
age += 1

male_indices = np.argwhere(gender == 1).flatten()
female_indices = np.argwhere(gender == 0).flatten()

lifetime = calculate_lifetime(L=Config.L, U = Config.U, fitness=fitness, age=age)
diversity = age[male_indices] / lifetime[male_indices]
population_diversity = diversity.mean()

In [90]:
%%time
female_preferred_age = np.array([FS.infer_partner_age(age=lifetime[i], diversity=population_diversity) for i in male_indices])

CPU times: user 2.57 s, sys: 0 ns, total: 2.57 s
Wall time: 2.57 s


In [108]:
female_preferred_age_1 = np.array([FS.infer_partner_age(age=lifetime[i], diversity=population_diversity) for i in male_indices])

In [109]:
(female_preferred_age_1 == female_preferred_age).all()

True

In [136]:
def infer_partner_ages(pth, ages, diversity, var, i):
    fs =  Inferrer(rule_path=pth)
    var.put((np.array([fs.infer_partner_age(age=a, diversity=diversity) for a in ages]), i))

In [173]:
def _extract_results(q: mp.Queue) -> np.array:
    """Extract results from multiprocessing queue

    Args:
        q (mp.Queue): multiprocessing queue

    Returns:
        np.array: female preferred age calculated
    """
    results = []
    
    # Simple dump of results to array
    while not q.empty():
        results.append(q.get())

    # Combining results into a single array with correct order
    results = sorted(results, key= lambda x: x[1])
    results = list(map(lambda x: x[0], results))
    return np.concatenate(results)

In [174]:
def multiprocessing_preferred_age(male_indices_to_reproduce: np.array, lifetime: np.array, population_diversity: float, N_proc: int = mp.cpu_count(),) -> np.array:
    splits = np.array_split(male_indices_to_reproduce, N_proc)
    ctx = mp.get_context('fork')
    processes = [i for i in range(N_proc)]

    # Launching 
    q = mp.Queue()
    for i, split in enumerate(splits):
        processes[i] = ctx.Process(target=infer_partner_ages, args=(Config.RULES_FILE, lifetime[split], population_diversity, q, i))
        processes[i].start()
    
    for i  in range(len(processes)):
        processes[i].join()
    
    return _extract_results(q)


In [175]:
result_parralel = multiprocessing_preferred_age(male_indices, lifetime, population_diversity)